In [4]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [5]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/Lenovo/Documents/ds_glassdoor/scraping-glassdoor-selenium-master/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.co.in/Job/us-data-analyst-jobs-SRCH_IL.0,2_IN1_KO3,15.htm'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(1)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
            print('selected : clicked')
        except ElementClickInterceptedException:
            pass

        time.sleep(10)

        try:
            driver.find_element_by_class_name("SVGInline-svg modal_closeIcon-svg").click()  #clicking to the X.
            print('close : clicked')
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            time.sleep(5)
            
            try:
                driver.find_element_by_class_name("SVGInline-svg modal_closeIcon-svg").click()  #clicking to the X.
                print('close : clicked')
            except NoSuchElementException:
                pass
        
            job_button.click()  #You might 
            time.sleep(5)

            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('//*[@id="HeroHeaderModule"]/div[3]/div[1]/div[4]/span').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()
                time.sleep(1.5)
                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [7]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data-analyst", 25,False)

selected : clicked
Progress: 0/25
Progress: 1/25
Progress: 2/25
Progress: 3/25
Progress: 4/25
Progress: 5/25
Progress: 6/25
Progress: 7/25
Progress: 8/25
Progress: 9/25
Progress: 10/25
Progress: 11/25
Progress: 12/25
Progress: 13/25
Progress: 14/25
Progress: 15/25
Progress: 16/25
Progress: 17/25
Progress: 18/25
Progress: 19/25
Progress: 20/25
Progress: 21/25
Progress: 22/25
Progress: 23/25
Progress: 24/25
Progress: 25/25


In [8]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Tactical Data Link (TDL) Analyst,$53K-$95K (Glassdoor Est.),Overview:\n\nOasis Systems has an exciting opp...,3.8,Oasis Systems LLC\n3.8,"Hampton, VA",-1,1001 to 5000 Employees,1997,Company - Private,IT Services,Information Technology,$100 to $500 million (USD),-1
1,Data Analyst,$33K-$57K (Glassdoor Est.),Job Details\nDescription\nMetroNet has an imme...,4.5,MetroNet\n4.5,"Evansville, IN",-1,1001 to 5000 Employees,2005,Company - Private,"Cable, Internet & Telephone Providers",Telecommunications,Unknown / Non-Applicable,-1
2,Senior Data Analyst,-1,Now Hiring a Senior Data Analyst\n\nInterested...,4.4,Applied Information Sciences\n4.4,Maryland,-1,501 to 1000 Employees,1982,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1
3,"Senior, Measurement Data Analyst",$65K-$112K (Glassdoor Est.),Position Summary\nPerforms data analysis to en...,3.5,Enable Midstream Partners\n3.5,"Oklahoma City, OK",-1,201 to 500 Employees,-1,Company - Public,Oil & Gas Exploration & Production,"Oil, Gas, Energy & Utilities",$10 to $25 million (USD),-1
4,Data Management Analyst,-1,What You Will Do:\nSupports the Member Contact...,-1,Security Service FCU,"San Antonio, TX",-1,-1,-1,-1,-1,-1,-1,-1


In [9]:
df.to_csv('glassdoor_scrape.csv')